<a href="https://colab.research.google.com/github/Priyanka2508/Singapore-Flat-Resale-Price-Prediction/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install streamlit
!pip install pickle

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstall

In [8]:
%%writefile app.py
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import warnings
import pickle
import streamlit as st
import re
import statistics
st.set_page_config(layout="wide")

st.title("INDUSTRIAL COPPER MODELLING")


tab1, tab2 = st.tabs(["ABOUT", "PREDICT"])

with tab1:

        st.markdown("# :blue[Singapore Resale Flat Prices Prediction]")
        st.markdown("### :blue[Overview :] This project aims to construct a machine learning model to predict resale price of the flat. "
                                          "This predictive model is based on historical data of resale flat transactions from 1990 to 2017 onwards, and it aims to assist both potential buyers and sellers in estimating the resale value of a flat.")
        st.markdown("### :blue[Model :] Decision Tree Regressor")
        st.markdown("### :blue[Model Accuracy :] 93.32%")
        st.markdown("### :blue[Features :] Town, Flat type, Flat model, Storey range, Floor area, leace Commence date(year), remaining years in lease")
        st.markdown("### :blue[Domain :] Real Estate")

with tab2:


        # Define the possible values for the dropdown menus
        town_options = ['SENGKANG','WOODLANDS','PUNGGOL','JURONG WEST','TAMPINES','YISHUN','BEDOK','HOUGANG','CHOA CHU KANG','ANG MO KIO','BUKIT MERAH','BUKIT PANJANG','BUKIT BATOK','TOA PAYOH',
        'PASIR RIS','KALLANG/WHAMPOA','QUEENSTOWN','SEMBAWANG','GEYLANG','CLEMENTI','JURONG EAST','SERANGOON','BISHAN','CENTRAL AREA','MARINE PARADE','BUKIT TIMAH']
        flat_model_options = ['Improved', 'New Generation', 'Model A', 'Standard', 'Simplified','Premium Apartment', 'Maisonette', 'Apartment', 'Model A2','Type S1', 'Type S2', 'Adjoined flat', 'Terrace', 'DBSS',
        'Model A-Maisonette', 'Premium Maisonette', 'Multi Generation','Premium Apartment Loft', 'Improved-Maisonette', '2-room', '3Gen']
        flat_type_options = ['3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE', '1 ROOM','MULTI-GENERATION']

        # Define the widgets for user input
        with st.form("my_form"):
            col1,col2,col3=st.columns([5,2,5])
            with col1:
                st.write(' ')
                town = st.selectbox("Town", town_options,key=1)
                flat_model = st.selectbox("Flat Model", flat_model_options,key=2)
                flat_type = st.selectbox("Flat Type", flat_type_options,key=3)

            with col3:
                floor_area_sqm = st.number_input('Floor Area (Per Square Meter)', min_value=1.0, max_value=500.0)
                lease_commence_year = st.number_input(('Lease Commence Year'), min_value = 1966, max_value = 2022)
                remaining_lease = st.number_input('Remaining Lease')
                storey_range = st.text_input("Storey Range (Format: 'Value1' TO 'Value2')")
                submit_button = st.form_submit_button(label="PREDICT RESALE PRICE")
                st.markdown("""
                    <style>
                    div.stButton > button:first-child {
                        background-color: #009999;
                        color: white;
                        width: 100%;
                    }
                    </style>
                """, unsafe_allow_html=True)

            flag=0
            pattern = "^(?:\d+|\d*\.\d+)$"
            for i in [floor_area_sqm,lease_commence_year,remaining_lease]:
                if re.match(pattern, str(i)):
                    pass
                else:
                    flag=1
                    break

        if submit_button and flag==1:
            if len(i)==0:
                st.write("please enter a valid number space not allowed")
            else:
                st.write("You have entered an invalid value: ",i)

        if submit_button and flag==0:

            import pickle
            with open(r"model.pkl", 'rb') as file:
                loaded_model = pickle.load(file)
            with open(r'scaler.pkl', 'rb') as f:
                scaler_loaded = pickle.load(f)
            with open(r"lc_town.pkl", 'rb') as f:
                lc_town = pickle.load(f)
            with open(r"lc_model.pkl", 'rb') as f:
                lc_model = pickle.load(f)
            with open(r"lc_type.pkl", 'rb') as f:
                lc_type = pickle.load(f)

            split_list = re.split(r'\sTO\s', storey_range)
            float_list = [float(i) for i in split_list]
            storey_median = statistics.median(float_list)
            new_sample= np.array([[town,flat_type,flat_model,int(lease_commence_year),int(remaining_lease),np.log(float(floor_area_sqm)),float(storey_median)]])
            new_sample[:, 0] = lc_town.transform(new_sample[:, 0])
            new_sample[:, 1] = lc_type.transform(new_sample[:, 1])
            new_sample[:, 2] = lc_model.transform(new_sample[:, 2])
            final_sample = scaler_loaded.transform(new_sample)
            new_pred = loaded_model.predict(final_sample)[0]
            st.write('## :green[Predicted selling price:] ', np.exp(new_pred))


Overwriting app.py
